In [1]:
import numpy as np
from getdata import load
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
from keras import backend as K

Using TensorFlow backend.


In [2]:
K.set_image_dim_ordering('th')

In [3]:
x_train, x_test, y_train, y_test = load()

In [20]:
len(x_train), len(x_test)

(1600, 400)

In [5]:
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [6]:
model = Sequential()
model.add(Convolution2D(32, kernel_size=(3, 3),padding='same',input_shape=(3 , 100, 100)))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('sigmoid'))

In [7]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.load_weights("weights.hdf5")

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 100, 100)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 100, 100)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 98, 98)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 98, 98)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 49, 49)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 49, 49)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 49, 49)        18496     
__________

In [9]:
out = model.predict_proba(x_test)
out = np.array(out)

400/400 [==============================] - 1s     


In [11]:
out

array([[  5.25076389e-02,   3.81866038e-01,   7.11544931e-01,
          3.03907059e-02,   2.52192646e-01],
       [  5.26888706e-02,   9.62114078e-04,   2.93579489e-01,
          9.72158492e-01,   5.14680594e-02],
       [  3.49882506e-02,   3.71032767e-03,   5.97689748e-01,
          9.70311522e-01,   2.65014381e-03],
       ..., 
       [  1.75322473e-01,   4.29682480e-03,   3.18148583e-01,
          9.09448683e-01,   3.30147594e-02],
       [  4.86278674e-03,   5.00628687e-02,   5.54790795e-01,
          3.92153813e-03,   4.14285213e-01],
       [  3.66323851e-02,   5.57558358e-01,   4.78139192e-01,
          7.09355948e-03,   2.37569943e-01]], dtype=float32)

In [12]:
threshold = np.arange(0.1,0.9,0.1)

acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append( matthews_corrcoef(y_test[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

In [13]:
best_threshold

array([ 0.6,  0.3,  0.5,  0.3,  0.7])

In [14]:
y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(y_test.shape[1])] for i in range(len(y_test))])

In [15]:
y_pred  #predicted labels

array([[0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0],
       ..., 
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0]])

In [16]:
y_test #actual labels

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       ..., 
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0]])

array([[0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0],
       ..., 
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0]])